In [83]:
import wandb
from pathlib import Path
import os
from tqdm.auto import tqdm
from PIL import Image
import numpy as np
import torch
from torch import nn
from datasets import load_dataset
from transformers import MaskFormerImageProcessor, MaskFormerForInstanceSegmentation
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchinfo import summary


In [65]:
os.environ['WANDB_NOTEBOOK_NAME']='EDA.ipynb'
os.environ["KMP_DUPLICATE_LIB_OK"]='TRUE'

In [3]:
Logging_Place= 'LOCAL'

if Logging_Place == 'LOCAL':
    os.environ['WANDB_BASE_URL']="http://10.24.1.19:8080"
    os.environ['WANDB_API_KEY']='local-96dd72cbb60b0155149e2f9dc985a636ffa77b28'
    ! wandb login --host=http://10.24.1.19:8080

elif Logging_Place =="CLOUD":
    os.environ['WANDB_BASE_URL']="https://api.wandb.ai"
    os.environ['WANDB_API_KEY']='be69ec2b537cb972b0106fabd867f0dc68c4468a'
    ! wandb login --host=https://api.wandb.ai

^C


wandb: Network error (HTTPError), entering retry loop.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [66]:
data_path=Path('COD10K-v2')
train_path=data_path / 'Train'
test_path= data_path / 'Test'

train_images_path= train_path / 'Images/Image'
train_labels_path= train_path / 'GT_Objects/GT_Object'

test_images_path= test_path / 'Images/Image'
test_labels_path= test_path / 'GT_Objects/GT_Object'

In [67]:
train_images_list=list(train_images_path.glob('*.jpg'))
print(train_images_list[0:5])
train_labels_list=list(train_labels_path.glob('*.png'))
print(train_labels_list[0:5])

[WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-1.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-3.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-7.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-8.jpg'), WindowsPath('COD10K-v2/Train/Images/Image/COD10K-CAM-1-Aquatic-1-BatFish-9.jpg')]
[WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-1.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-3.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-7.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-8.png'), WindowsPath('COD10K-v2/Train/GT_Objects/GT_Object/COD10K-CAM-1-Aquatic-1-BatFish-9.png')]


In [68]:
super_classes_list=[ i.stem.split('-')[3] for i in train_images_list]
super_classes_list=[*set(super_classes_list)]
print(super_classes_list)


['Other', 'Aquatic', 'Terrestial', 'Background', 'Amphibian', 'Flying', 'Terrestrial']


In [69]:
index=5
sub_classes_list=[]
for i in train_images_list:
    split_path=i.stem.split('-')
    count=len(split_path)
    if (index<count):
        sub_classes_list.append(split_path[index])
sub_classes_list=[*set(sub_classes_list)]
print(sub_classes_list)

['Lizard', 'Shrimp', 'Beetle', 'Cat', 'Monkey', 'Katydid', 'Sciuridae', 'Sand', 'Sheep', 'Toad', 'Sky', 'Bittern', 'Bug', 'Cheetah', 'Wolf', 'Dragonfly', 'Bird', 'Indoor', 'SeaHorse', 'Grouse', 'Ocean', 'CrocodileFish', 'Flounder', 'Turtle', 'StickInsect', 'Leopard', 'Frog', 'Vegetation', 'Gecko', 'Reccoon', 'Slug', 'Dog', 'LeafySeaDragon', 'StarFish', 'Duck', 'BatFish', 'Frogmouth', 'Mantis', 'Mockingbird', 'ClownFish', 'Fish', 'Deer', 'Octopus', 'Crocodile', 'GhostPipefish', 'Spider', 'Stingaree', 'Ant', 'Cicada', 'Heron', 'Owl', 'Bee', 'Centipede', 'Owlfly', 'Chameleon', 'Other', 'Rabbit', 'Lion', 'Bat', 'Pipefish', 'FrogFish', 'Tiger', 'Butterfly', 'Grasshopper', 'ScorpionFish', 'Kangaroo', 'Worm', 'Caterpillar', 'Moth', 'Snake', 'Giraffe', 'Pagurian', 'Crab', 'Human']


#####    Setting up dataloaders

In [70]:
class CamouflageDataset(Dataset):
    def __init__(
        self,
        image_directory:str,
        gt_directory:str,
        transform=None
    ):
        self.image_directory=image_directory
        self.gt_directory=gt_directory
        self.transforms = transform
        self.image_list=list(self.image_directory.glob('*.jpg'))
        self.gt_list=list(self.gt_directory.glob('*.png'))

    def __getitem__(self, idx):
        img=Image.open(self.image_list[idx])
        mask=Image.open(self.gt_list[idx])
        self.transforms = transforms.Compose([transforms.ToTensor()])
        img=self.transforms(img)
        mask=self.transforms(mask)     
        return img, mask

    def __len__(self):
        return len(self.image_list)

In [71]:
train_dataset=CamouflageDataset(image_directory=train_images_path,
                                gt_directory=train_labels_path
                                )
test_dataset=CamouflageDataset(image_directory=test_images_path,
                                gt_directory=test_labels_path
                               )

In [73]:
train_dataloader=DataLoader(train_dataset,batch_size=1, shuffle=True)
test_dataloader=DataLoader(test_dataset,batch_size=1, shuffle=False)

In [74]:
train_features, train_labels = next(iter(test_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1, 3, 533, 800])
Labels batch shape: torch.Size([1, 1, 533, 800])


In [76]:
print(len(train_dataset))
print(len(test_dataset))

6000
4000


In [78]:
from transformers import SegformerFeatureExtractor
from transformers import SegformerForSemanticSegmentation

In [82]:
model = SegformerForSemanticSegmentation.from_pretrained("mit-b0")

Some weights of the model checkpoint at mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at mit-b0 and are newly initialized: ['decode_head.linear_c.3.proj.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.0.proj.bias', 'decode_head.batch_norm.running

In [84]:
summary(model=model)

Layer (type:depth-idx)                                                      Param #
SegformerForSemanticSegmentation                                            --
├─SegformerModel: 1-1                                                       --
│    └─SegformerEncoder: 2-1                                                --
│    │    └─ModuleList: 3-1                                                 485,472
│    │    └─ModuleList: 3-2                                                 2,832,896
│    │    └─ModuleList: 3-3                                                 1,024
├─SegformerDecodeHead: 1-2                                                  --
│    └─ModuleList: 2-2                                                      --
│    │    └─SegformerMLP: 3-4                                               8,448
│    │    └─SegformerMLP: 3-5                                               16,640
│    │    └─SegformerMLP: 3-6                                               41,216
│    │    └─Segformer